# Portfolio construction example

- Trade several spot DEX tokens
    - We use a predefined set of tokens for Polygon
    - You could e.g. load this list from a CSV file
- Create a portfolio of tokens based on their momentum
    - [See the full blog post about what's portfolio construction](https://tradingstrategy.ai/blog/writing-portfolio-construction-strategy-in-python)
    - We use daily RSI as the alpha signal for the portfolio construction
    - This strategy does not try to be meaningful, but just sets up a skeleton for a portfolio construction strategy
    - You can figure out how to plug in your own signals
- This backtest does not consider liquidity or price impact 
    - Adding tokens with non-tradeable liquidity is likely to break the backtest

# Set up

Set up Trading Strategy data client.


In [ ]:
from tradeexecutor.utils.notebook import setup_charting_and_output
from tradingstrategy.client import Client
from tradeexecutor.utils.notebook import setup_charting_and_output, OutputMode

client = Client.create_jupyter_client()

# Set up drawing charts in interactive vector output mode.
# This is slower. See the alternative commented option below.
# Kernel restart needed if you change output mode.
# setup_charting_and_output(OutputMode.interactive)

# Set up rendering static PNG images.
# This is much faster but disables zoom on any chart.
setup_charting_and_output(OutputMode.static, image_format="png", width=1500, height=1000)


# Parameters

- Strategy parameters define the fixed and grid searched parameters

In [ ]:
from tradingstrategy.chain import ChainId
import datetime

from tradeexecutor.strategy.default_routing_options import TradeRouting
from tradingstrategy.timebucket import TimeBucket
from tradeexecutor.strategy.cycle import CycleDuration
from tradeexecutor.strategy.parameters import StrategyParameters


class Parameters:
    """Parameteres for this strategy.

    - Collect parameters used for this strategy here

    - Both live trading and backtesting parameters
    """

    id = "portfolio-construction" # Used in cache paths

    cycle_duration = CycleDuration.d1  # Daily rebalance
    candle_time_bucket = TimeBucket.d1  
    allocation = 0.98   
    
    max_assets = 4  # How many assets hold in portfolio once
    rsi_length = 7   # Use 7 days RSI as the alpha signal for a trading pair
    minimum_rebalance_trade_percent = 0.25  # Portfolio must change at least 1/4 before we start doing rebalance trades

    #
    # Live trading only
    #
    chain_id = ChainId.polygon
    routing = TradeRouting.default  # Pick default routes for trade execution
    required_history_period = datetime.timedelta(days=rsi_length + 1)

    #
    # Backtesting only
    #
    backtest_start = datetime.datetime(2023, 1, 1)
    backtest_end = datetime.datetime(2024, 1, 1)
    initial_cash = 10_000


parameters = StrategyParameters.from_class(Parameters)  # Convert to AttributedDict to easier typing with dot notation


# Trading pairs and market data

- Get a list of ERC-20 tokens we are going to trade on Polygon
- Trading pairs are automatically mapped to the best volume /USDC or /WMATIC pair
    - Limited to current market information - no historical volume/liquidity analyses performed here
- This data loading method caps out at 75 trading pairs

In [ ]:
from tradingstrategy.client import Client

from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse, load_partial_data
from tradeexecutor.strategy.execution_context import ExecutionContext, notebook_execution_context
from tradeexecutor.utils.binance import create_binance_universe
from tradeexecutor.strategy.universe_model import UniverseOptions
from tradeexecutor.strategy.pandas_trader.token_mapper import TokenTuple, create_trading_universe_for_tokens

# I have no idea what tokens these are,
# but they are on Polygon and trade spot DEXes there.
# The last token 0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270 is WMATIC and we use WMATIC/USDC as the portfolio benchmark index.
# Most of these assets are badly broken e.g. only appear for one day, 
# or otherwise bad data.
POLYGON_ERC20_ADDRESS_LIST = """
0xc2132d05d31c914a87c6611c10748aeb04b58e8f
0xf28164a485b0b2c90639e47b0f377b4a438a16b1
0xe04830e66539867df44fa95094e90a2006681bb6
0x9aba994219c08440a52dea5e43ff51f339aff2bc
0xaac532deb0878e94f3fde500996a0d32be4ffdc5
0x0b6afe834dab840335f87d99b45c2a4bd81a93c7
0x04f177fcacf6fb4d2f95d41d7d3fee8e565ca1d0
0xb2c63830d4478cb331142fac075a39671a5541dc
0x6002410dda2fb88b4d0dc3c1d562f7761191ea80
0x431cd3c9ac9fc73644bf68bf5691f4b83f9e104f
0x9c891326fd8b1a713974f73bb604677e1e63396d
0x2760e46d9bb43dafcbecaad1f64b93207f9f0ed7
0x5ec03c1f7fa7ff05ec476d19e34a22eddb48acdc
0xb7486718ea21c79bbd894126f79f504fd3625f68
0x8a16d4bf8a0a716017e8d2262c4ac32927797a2f
0x483dd3425278c1f79f377f1034d9d2cae55648b6
0x58d70ef99a1d22e1a8f8f0e8f27c1babcf8464f3
0xb58458c52b6511dc723d7d6f3be8c36d7383b4a8
0x91b2745d7aca9d64560cd1693b6ff96678ffc433
0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270
"""


def create_trading_universe(
    timestamp: datetime.datetime,
    client: Client,
    execution_context: ExecutionContext,
    universe_options: UniverseOptions,
) -> TradingStrategyUniverse:
    """Create the trading universe."""

    # Build a list of unique tokens
    polygon_token_list = {TokenTuple(ChainId.polygon, address) for address in POLYGON_ERC20_ADDRESS_LIST.strip().split()}

    strategy_universe = create_trading_universe_for_tokens(
        client,
        execution_context,
        universe_options,
        Parameters.candle_time_bucket,
        polygon_token_list,
        reserve_token="0x2791bca1f2de4661ed88a30c99a7a9449aa84174",  # USDC.e bridged on Polygon
        intermediate_token="0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270",  # WMATIC on Polygon
    )
    return strategy_universe


strategy_universe = create_trading_universe(
    None,
    client,
    notebook_execution_context,
    UniverseOptions.from_strategy_parameters_class(Parameters, notebook_execution_context)
)

broken_trading_pairs = set()

#
# Extra sanity checks
# 
# Ru some extra sanity check for small cap tokens
#

print("Checking trading pair quality")
print("-" * 80)

for pair in strategy_universe.iterate_pairs():
    reason = strategy_universe.get_trading_broken_reason(pair, min_candles_required=Parameters.rsi_length)
    if reason:
        print(f"FAIL: {pair} with base token {pair.base.address} may be problematic: {reason}")
        broken_trading_pairs.add(pair)
    else:
        print(f"OK: {pair} included in the backtest")

print(f"Total {len(broken_trading_pairs)} broken trading pairs detected")


# Indicators

- We use `pandas_ta` Python package to calculate technical indicators
- These indicators are precalculated and cached on the disk

In [ ]:
import pandas as pd
import pandas_ta

from tradeexecutor.analysis.regime import Regime
from tradeexecutor.strategy.execution_context import ExecutionContext
from tradeexecutor.strategy.pandas_trader.indicator import IndicatorSet, IndicatorSource
from tradeexecutor.strategy.parameters import StrategyParameters
from tradeexecutor.strategy.trading_strategy_universe import TradingStrategyUniverse
from tradingstrategy.utils.groupeduniverse import resample_candles


def rsi_safe(close_series: pd.Series, length: int):
    # Work around tokens that appear only for few hours*
    if len(close_series) > length:
        return pandas_ta.rsi(close_series, length)
    else:
        # The token did not trade long enough we could have ever calculated RSI
        return pd.Series(dtype="float64", index=pd.DatetimeIndex([]))


def create_indicators(
    timestamp: datetime.datetime | None,
    parameters: StrategyParameters,
    strategy_universe: TradingStrategyUniverse,
    execution_context: ExecutionContext
):
    indicators = IndicatorSet()

    indicators.add(
        "rsi",
        rsi_safe,
        {"length": parameters.rsi_length},
        IndicatorSource.close_price,
    )        
    return indicators


# Trading algorithm

- Describe out trading strategy as code

In [ ]:
from tradeexecutor.state.visualisation import PlotKind
from tradeexecutor.state.trade import TradeExecution
from tradeexecutor.strategy.pandas_trader.strategy_input import StrategyInput
from tradeexecutor.strategy.weighting import weight_by_1_slash_n
from tradeexecutor.strategy.alpha_model import AlphaModel
from tradeexecutor.strategy.pandas_trader.strategy_input import IndicatorDataNotFoundWithinDataTolerance


def decide_trades(
    input: StrategyInput,
) -> list[TradeExecution]:

    # 
    # Decision cycle setup.
    # Read all variables we are going to use for the decisions.
    #
    parameters = input.parameters
    position_manager = input.get_position_manager()
    state = input.state
    timestamp = input.timestamp
    indicators = input.indicators
    strategy_universe = input.strategy_universe

    cash = position_manager.get_current_cash()

    # Another low cap problem checker.
    # Doing a bad non-liquidity trade may break the valuation calculations.
    total_equity = state.portfolio.get_total_equity()
    if total_equity > 1_000_000:
        position_valuations = "\n".join([f"{p} (token {p.pair.base.address}): {p.get_value()}" for p in state.portfolio.open_positions.values()])
        raise RuntimeError(f"Portfolio total equity exceeded 1,000,000 USD. Some broken math likely happened. Total equity is {total_equity} USD.\nOpen positions:\n{position_valuations}")
        
    #
    # Trading logic
    #
    # We do some extra checks here as we are trading low quality
    # low cap tokens which often have outright malicious data for trading.
    #

    alpha_model = AlphaModel(timestamp)

    trades = []
    for pair in strategy_universe.iterate_pairs():

        if pair in broken_trading_pairs:
            # Don't even bother to try trade this
            continue

        # Check last close price
        price = indicators.get_price(pair=pair)
        if (price is None) or (price < 0.000001):
            # This pair has its price range moved to buggy floating point range,
            # avoid            
            continue

        position_manager.log(f"Pricing check. Pair {pair}, price {price}")

        try:        
            rsi = indicators.get_indicator_value("rsi", pair=pair)        
        except IndicatorDataNotFoundWithinDataTolerance:
            # Data not yet availble, too spotty
            rsi = None

        # Does the trading pair have good RSI in this point of history?
        if rsi is not None:  
            # Interpret RSI above +50 as a positive momentum        
            signal = rsi - 50
            if signal > 0:
                # Consider positive signals only, because we cannot short
                alpha_model.set_signal(pair, signal)

    # Use alpha model and construct a portfolio of four top assets
    alpha_model.select_top_signals(parameters.max_assets)
    alpha_model.assign_weights(weight_by_1_slash_n)
    alpha_model.normalise_weights()
    alpha_model.update_old_weights(state.portfolio)
    portfolio = position_manager.get_current_portfolio()
    portfolio_target_value = portfolio.get_total_equity() * parameters.allocation
    alpha_model.calculate_target_positions(position_manager, portfolio_target_value)
    trades = alpha_model.generate_rebalance_trades_and_triggers(
        position_manager,
        min_trade_threshold=parameters.minimum_rebalance_trade_percent * portfolio.get_total_equity(),
    )

    # Visualisations
    #
    if input.is_visualisation_enabled():
        visualisation = state.visualisation
        state.visualisation.add_calculations(timestamp, alpha_model.to_dict())  # Record alpha model thinking

    return trades  # Return the list of trades we made in this cycle

# Backtest

- Run the backtest

In [ ]:
import logging
from tradeexecutor.backtest.backtest_runner import run_backtest_inline

result = run_backtest_inline(
    name=parameters.id,
    engine_version="0.5",
    decide_trades=decide_trades,
    create_indicators=create_indicators,
    client=client,
    universe=strategy_universe,
    parameters=parameters,
    strategy_logging=False,
    max_workers=1,
    # We need to set this really high value, because
    # some low cap tokens may only see 1-2 trades per year
    # and our backtesting framework aborts if it thinks
    # there is an issue with data quality
    data_delay_tolerance=pd.Timedelta(days=365),
    
    # Uncomment to enable verbose logging
    # log_level=logging.INFO,
)

state = result.state

trade_count = len(list(state.portfolio.get_all_trades()))
print(f"Backtesting completed, backtested strategy made {trade_count} trades")

# Equity curve

- Equity curve shows how your strategy accrues value over time
- A good equity curve has a stable ascending angle
- Benchmark against MATIC buy and hold

In [ ]:
import pandas as pd
from tradeexecutor.analysis.multi_asset_benchmark import get_benchmark_data
from tradeexecutor.visual.benchmark import visualise_equity_curve_benchmark

# Pulls WMATIC/USDC as the benchmark
benchmark_indexes = get_benchmark_data(
    strategy_universe,
    cumulative_with_initial_cash=state.portfolio.get_initial_cash()
)

fig = visualise_equity_curve_benchmark(
    name=state.name,
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_cash(),
    benchmark_indexes=benchmark_indexes,
    height=800,
    log_y=True,
)

fig.show()

# Performance metrics

- Display portfolio performance metrics
- Compare against buy and hold matic using the same initial capital

In [ ]:
from tradeexecutor.analysis.multi_asset_benchmark import compare_strategy_backtest_to_multiple_assets

compare_strategy_backtest_to_multiple_assets(
    state,
    strategy_universe,
    display=True,
)

# Trading statistics

- Display summare about made trades

In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)
summary = analysis.calculate_summary_statistics()
display(summary.to_dataframe())

# Alpha model thinking

- Show the portfolio construction steps for each decision cycle
- Render a table where we show what was the signal for each asset and how much it was bought or sod

In [ ]:
from IPython.display import HTML

from tradeexecutor.analysis.alpha_model_analyser import render_alpha_model_plotly_table, create_alpha_model_timeline_all_assets

# Render alpha model timeline as Pandas HTML table
timeline = create_alpha_model_timeline_all_assets(state, strategy_universe, new_line="CcC")
HTML(timeline.to_html().replace("CcC", "<br>"))


# Alternative Plotly renderer,
# does not work in notebook HTML export but has richer output
#figure, table = render_alpha_model_plotly_table(timeline)